In [22]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import joblib
import imodelsx.process_results
import sys
from sklearn.metrics import accuracy_score

sys.path.append('../experiments/')
results = []

results_dir = f'../results/distillation_cub/cub'
experiment_filename = '../experiments/distillation_cub.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# save results as csvs for backup
r.to_csv('../csv_results/distillation_cub.csv')

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 732.97it/s]


TypeError: unhashable type: 'list'

In [21]:
r

,save_dir,teacher_path,train_path,test_path,task_type,student_name,n_trees_list,n_rules_list,n_depth_list,min_impurity_decrease_list,...,student_prediction_rand_interv_train_accuracy,student_prediction_rand_interv_test_accuracy,teacher_prediction_rand_interv_train_accuracy,teacher_prediction_rand_interv_test_accuracy,student_distillation_lin_interv_train_accuracy,student_distillation_lin_interv_test_accuracy,student_prediction_lin_interv_train_accuracy,student_prediction_lin_interv_test_accuracy,teacher_prediction_lin_interv_train_accuracy,teacher_prediction_lin_interv_test_accuracy
0,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_MLP1_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,"[30, 40]",[200],[4],[0],...,0.939672,0.752503,0.961731,0.797031,NaN,NaN,NaN,NaN,NaN,NaN
1,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_Linear_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,"[30, 40]",[200],[4],[0],...,0.954211,0.770280,0.971758,0.818778,0.95488,0.850535,0.951537,0.759061,0.967246,0.796168
2,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_MLP2_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,"[30, 40]",[200],[4],[0],...,0.926638,0.738868,0.957386,0.779082,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
r.loc[:, ~r.columns.str.contains('train')&~r.columns.str.contains('prediction')].round(4)

,save_dir,teacher_path,test_path,task_type,student_name,n_trees_list,n_rules_list,n_depth_list,min_impurity_decrease_list,max_rules,...,gpu,use_cache,save_dir_unique,n_trees,n_rules,student_distillation_test_accuracy,depth,student_distillation_adap_interv_test_accuracy,student_distillation_rand_interv_test_accuracy,student_distillation_lin_interv_test_accuracy
0,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_MLP1_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,"[30, 40]",[200],[4],[0],200,...,1,1,/home/mattyshen/DistillationEdit/results/disti...,30,200,0.8405,3,0.8120,0.8186,NaN
1,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_Linear_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,"[30, 40]",[200],[4],[0],200,...,0,1,/home/mattyshen/DistillationEdit/results/disti...,30,200,0.8597,3,0.8548,0.8514,0.8505
2,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_MLP2_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,"[30, 40]",[200],[4],[0],200,...,2,1,/home/mattyshen/DistillationEdit/results/disti...,30,200,0.8329,3,0.7988,0.8019,NaN


In [23]:
r['teacher_prediction_adap_interv_test_accuracy'] - r['teacher_prediction_test_accuracy']

0    0.026234
1    0.040904
2    0.031067
dtype: float64

In [24]:
r['teacher_prediction_lin_interv_test_accuracy'] - r['teacher_prediction_test_accuracy']

0         NaN
1   -0.002244
2         NaN
dtype: float64

In [25]:
r['teacher_prediction_rand_interv_test_accuracy'] - r['teacher_prediction_test_accuracy']

0    0.007249
1    0.020366
2   -0.001036
dtype: float64